<a href="https://colab.research.google.com/github/pickledherring/NLP-group/blob/main/gloss_glove_sgns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from googleapiclient.discovery import build
from io import FileIO
from googleapiclient.http import MediaIoBaseDownload

import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

try:
  stop_words = nltk.corpus.stopwords.words("english")
except:
  nltk.download('stopwords')
  stop_words = nltk.corpus.stopwords.words("english")

import numpy as np
import torch
import random
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#  Get the files from google drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Get english train data file
file_id = '1m3Ax9Z8OHMU-7FqraKc-ddI3YQ7yY_Q6'  # file id on the Google Drive
downloaded = FileIO("en.trial.complete.json", 'w')
request = drive_service.files().get_media(fileId=file_id)
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download {}%.".format(int(status.progress() * 100)))


Download 100%.


In [3]:
en_df = pd.read_json("en.trial.complete.json")
en_df.head()

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
0,en.trial.1,beautiful,adjective,Pleasant ; clear .,"It 's beautiful outside , let 's go for a walk .",synonym/antonym-based,124908,706,0,0,"[1.393769145, 0.7516670227000001, -2.581333160...","[0.295645088, 0.098426342, 0.0463486575, 0.016...","[0.0800914839, -0.1875839084, -0.0411579385000..."
1,en.trial.2,cocktail,noun,A mixture of other substances or things .,a cocktail of illegal drugs,hypernym-based,4187,13245,1,0,"[2.0872907639, 0.2617726326, 0.668431639700000...","[0.3878918886, 0.1971583217, -0.44026631120000...","[-1.4771454334, -0.4742421806, 0.0847439319, -..."
2,en.trial.3,institutionalized,adjective,Having been established as an institution .,It is very difficult to get bureaucracies to a...,paraphrastic,961,35934,0,0,"[0.7893871069, -0.43510755900000003, 0.8553860...","[-0.0519028902, 0.2257766128, -0.1839749813, 0...","[-1.1030955315, -0.9046602845, 0.1503403783, -..."
3,en.trial.4,menial,noun,"A servant , especially a domestic servant .","The world was awake to the 2nd of May , but Ma...",hypernym-based,517,53267,1,1,"[0.1222261563, 0.1572209597, 0.5396134257, -0....","[-0.3667449057, -0.1431699395, -0.0671329796, ...","[-1.6584062576, -0.24498166140000002, 0.150174..."
4,en.trial.5,seek,verb,To try to find ; to look for ; to search for .,"Not long ago , it was difficult to produce pho...",paraphrastic,25195,3212,0,0,"[1.1894155741, 1.3668279648000001, -1.61634504...","[0.6137102246, 0.5464909673, -0.0161557049, 9....","[-0.5474479198000001, -0.0880863219, 0.0784259..."


In [4]:
en_df.dtypes

id            object
word          object
pos           object
gloss         object
example       object
type          object
counts         int64
f_rnk          int64
concrete       int64
polysemous     int64
sgns          object
char          object
electra       object
dtype: object

In [5]:
en_df[en_df.word.duplicated()]

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
66,en.trial.67,divorce,verb,To legally dissolve a marriage between two peo...,"A ship captain can marry couples , but can not...",paraphrastic,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[0.0899147466, 0.0271891933, 0.023069024100000..."
188,en.trial.189,forthwith,adverb,Without delay ; immediately .,"Then Proclamation was made , that they that ha...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-1.0183763504, -0.3864063621, 0.0048319194, -..."


we have words to disambiguate! forthwith looks to be the same with a different example though. <br>thoughts on disambiguation (feel free to add):<br><br> need to gather contex, but all I can think of is to train the NN to find the part of speech of the word and maybe type. I think we are only given the gloss and from that predict the embedding, so we (maybe) can't use the rest of it except in training. probably a good idea regardless of the disambiguation goal. <br><br> we could also retrain the model on incorrect words that are polysemous, like a boosting method. <br><br> lastly, disambiguation might not be necessary given the gloss and the vector might not be so similar. should look at the distance between these words in the embeddings. glosses look different, but I can see the top few elements of these two polysemous words are similar in all of the embeddings.

In [6]:
en_df[(en_df.word == "divorce") | (en_df.word == "forthwith")]

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
50,en.trial.51,divorce,verb,To separate something that was connected .,The radical group voted to divorce itself from...,hypernym-based,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[-0.6520454288, -0.1928912848, 0.0298318155, -..."
66,en.trial.67,divorce,verb,To legally dissolve a marriage between two peo...,"A ship captain can marry couples , but can not...",paraphrastic,21171,3773,0,0,"[1.579477787, -1.7040250301, 2.4623465538, -0....","[-0.09191438560000001, -0.0440451615, -0.45280...","[0.0899147466, 0.0271891933, 0.023069024100000..."
160,en.trial.161,forthwith,adverb,Without delay ; immediately .,"Let ther be Light , said God , and forthwith L...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-0.9691627026, -0.20675842460000002, 0.184671..."
188,en.trial.189,forthwith,adverb,Without delay ; immediately .,"Then Proclamation was made , that they that ha...",synonym/antonym-based,1179,31462,0,1,"[0.7841670513, 0.6406752467, -1.0482375622, -0...","[-8.645650000000001e-05, 0.161273554, 0.082766...","[-1.0183763504, -0.3864063621, 0.0048319194, -..."


clean glosses of punctuation, stopwords, duplicates, turn into lists.

In [7]:
def clean(gloss):
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  cleaned = tokenizer.tokenize(gloss)
  cleaned = list(set([word.lower() for word in cleaned]))
  cleaned = [word for word in cleaned if not word in stop_words]
  return cleaned

gloss_lists = en_df.gloss.apply(clean)
gloss_lists

0                                      [clear, pleasant]
1                          [mixture, things, substances]
2                             [established, institution]
3                        [servant, domestic, especially]
4                              [try, look, search, find]
                             ...                        
195                        [plant, cells, animal, color]
196                                [appearance, seeming]
197       [vehicle, travel, permission, proceed, person]
198                      [moving, around, much, sitting]
199    [word, forming, achievement, heraldic, sentenc...
Name: gloss, Length: 200, dtype: object

In [8]:
# list of all context words
context_voc = []
for i in range(len(gloss_lists)):
  for j in range(len(gloss_lists[i])):
    if not gloss_lists[i][j] in context_voc:
      context_voc.append(gloss_lists[i][j])

based on an SGNS following this guide: https://medium.com/towards-datascience/word2vec-negative-sampling-made-easy-7a1a647e07a4


In [9]:
# true context words for each defined word (center word)
trues = []
for i in range(len(gloss_lists)):
  for j in range(len(gloss_lists[i])):
    index = context_voc.index(gloss_lists[i][j])
    # append index of center in gloss_lists, index of context in context_voc, and 1 for true
    trues.append([i, index, 1])

In [10]:
trues[:5]

[[0, 0, 1], [0, 1, 1], [1, 2, 1], [1, 3, 1], [1, 4, 1]]

this could be improved by weighting the selection by the count of the word raised to 3/4 over the sum of the counts of all words raised to 3/4.

we can also try resampling with each epoch

In [11]:
falses = []
# create 3 randomly sampled context words for each true context word
# these may be true, but probably not. we'll label them false
for i in range(len(trues)):
  for j in range(3):
    center_index = trues[i][0]
    context_index = random.sample(range(len(context_voc)), 1)[0]
    falses.append([center_index, context_index, 0])

In [15]:
def gen_onehot():
  # combine and shuffle trues and falses
  together = np.concatenate((np.array(trues), np.array(falses)))
  np.random.shuffle(together)
  targets = torch.as_tensor(together)

  # matrices to one hot encode middle words and target words
  middle_tensor = torch.zeros(targets.shape[0], gloss_lists.shape[0])
  context_tensor = torch.zeros(targets.shape[0], len(context_voc))

  for i in range(middle_tensor.shape[0]):
    middle_tensor[i, targets[i, 0]] = 1
    context_tensor[i, targets[i, 1]] = 1

  labels = targets[:, 2].float()
  return (middle_tensor, context_tensor, labels)

build the model itself below. not sure why bias needs to be false, should try as true.

this would be more elegant as a class or at least a function combined with the forward (dot product production) part included.

In [19]:
# fully connected middle layers for middle and target words
mid_fc = torch.nn.Linear(gloss_lists.shape[0], 256, bias = False)
con_fc = torch.nn.Linear(len(context_voc), 256, bias = False)
sig = torch.nn.Sigmoid()
params = list(mid_fc.parameters()) + list(con_fc.parameters())
optim = torch.optim.Adam(params, lr = .001)
loss_fn = torch.nn.BCELoss()

In [20]:
epochs = 100
mid_hot, con_hot, labels = gen_onehot()
for i in range(epochs):
  trans_mid = mid_fc(mid_hot)
  trans_con = con_fc(con_hot)
  dot_mat = torch.zeros(mid_hot.shape[0], 1)
  # for each row dot a center embedding by a target embedding
  for j in range(len(trans_mid)):
    dot_mat[j, :] = trans_mid[j, :] @ trans_con[j, :]
  # sigmoid transformation, then compute the gradient and backwards propagate
  prob_mat = sig(dot_mat)
  optim.zero_grad()
  loss = loss_fn(prob_mat, torch.Tensor(labels).view(prob_mat.shape[0], 1))
  loss.backward()
  optim.step()

  # print loss every 10 epochs
  if i % 10 == 0:
    print(loss.data)
  mid_hot, con_hot, labels = gen_onehot()

tensor(0.6932)
tensor(0.6690)
tensor(0.6393)
tensor(0.5999)
tensor(0.5495)
tensor(0.4892)
tensor(0.4224)
tensor(0.3541)
tensor(0.2894)
tensor(0.2323)


In [26]:
print(loss.data)

tensor(0.1893)


array([-0.1411513 , -0.11774216, -0.10432833, -0.1687036 ,  0.1566767 ,
       -0.11107994, -0.12471368,  0.15332888,  0.15980025, -0.13986713,
        0.12381111, -0.18713355,  0.07512406,  0.08450958, -0.13047186,
       -0.12404388, -0.03525103,  0.07027913, -0.16128404,  0.17018603,
        0.15817115,  0.16028467, -0.07011227,  0.16349585, -0.11943942,
       -0.13224588,  0.01074382, -0.14261241,  0.14661738,  0.0774352 ,
       -0.08099443, -0.04730928, -0.16103853,  0.1803701 ,  0.1545418 ,
       -0.08406659, -0.1280773 , -0.15287724,  0.09027255,  0.13854425,
        0.12404647,  0.17152335, -0.10203484, -0.17817739, -0.15218218,
        0.06690591,  0.08689253,  0.1528855 , -0.17611659,  0.15164353,
        0.10886443,  0.15343897, -0.16868179,  0.14526094,  0.06567025,
       -0.17030992,  0.0192152 , -0.12299297, -0.10162177,  0.1290196 ,
       -0.08774319,  0.10496914,  0.1691124 , -0.15756284, -0.16245049,
        0.06867835, -0.11665383, -0.06564275,  0.1681042 ,  0.09

In [27]:
embeddings = mid_fc.weight.t().detach().numpy()
X_train, X_test, y_train, y_test = train_test_split(embeddings, en_df.word)

array([[ 0.15594737,  0.16757499, -0.15409352, ..., -0.08538463,
        -0.11705005, -0.12314182],
       [ 0.11836023,  0.18111967,  0.16036275, ..., -0.08697964,
        -0.13469538,  0.1456157 ],
       [ 0.1762775 , -0.17557818, -0.11354394, ...,  0.16546732,
         0.1399769 ,  0.14017747],
       ...,
       [-0.14293912,  0.18295081,  0.03600238, ...,  0.14055118,
        -0.1383612 ,  0.18642117],
       [-0.1411513 , -0.11774216, -0.10432833, ..., -0.06486448,
         0.16163446, -0.12581474],
       [ 0.10709661,  0.15535395, -0.06540873, ..., -0.18325497,
        -0.15104418, -0.14411506]], dtype=float32)

In [29]:
print(X_test.shape, "\n", y_test.shape)

(50, 256) 
 (50,)
